#Aim

Given a large set of sequences or graphs with ordered vertices find small vertex ordered subsequences that are most discriminative for the set.

Steps:
- devise a negative set
- learn a discriminative model
- annotate importance on vertices
- extract max subarrays 
- cluster them 
 - use fast EDeN string kernel 
 - custering algorithm
 
Output: 
1. all sequence motives in each cluster
2. all initial sequences with motif location (begin,end) and cluster id (build regex from all seqs in cluster and run a find iterator)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

In [3]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGU', motives=None, motif_length=6, sequence_length=100, n_sequences=1000, n_motives=2, p=0.2):
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
        
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences / n_motives

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            seqs.append(('>ID%d'%counter,seq))
            counter += 1
    return motives, seqs

In [4]:
from eden.motif import SequenceMotif
help(SequenceMotif)

Help on class SequenceMotif in module eden.motif:

class SequenceMotif(__builtin__.object)
 |  Methods defined here:
 |  
 |  __init__(self, min_subarray_size=7, max_subarray_size=10, min_motif_count=1, min_cluster_size=1, training_size=None, negative_ratio=2, shuffle_order=2, n_iter_search=1, complexity=4, nbits=20, clustering_algorithm=None, n_jobs=4, n_blocks=8, block_size=None, pre_processor_n_jobs=4, pre_processor_n_blocks=8, pre_processor_block_size=None, random_state=1)
 |  
 |  fit(self, seqs, neg_seqs=None)
 |      Builds a discriminative estimator.
 |      Identifies the maximal subarrays in the data.
 |      Clusters them with the clustering algorithm provided in the initialization phase.
 |      For each cluster builds a fast sequence search model (Aho Corasick data structure).
 |  
 |  fit_predict(self, seqs, return_list=False)
 |  
 |  fit_transform(self, seqs, return_match=False)
 |  
 |  load(self, obj)
 |  
 |  predict(self, seqs, return_list=False)
 |      Returns for

#Experimental Setup

In [5]:
#setup parameters
alphabet='ACGU'
motives=['AAAAAAAAAA','CCCCCCCCCC','GGGGGGGGGG','UUUUUUUUUU']
sequence_length=300
n_sequences=40
p=0.3

#make dataset
motives, seqs = make_artificial_dataset(alphabet=alphabet,motives=motives,sequence_length=sequence_length,n_sequences=n_sequences,p=p)

#display
print 'Motives and sample of their perturbed variants:'
alphabet_list=[c for c in alphabet]
for motif in motives: 
    print
    print motif,
    for i in range(9):
        print perturb(motif,alphabet_list,p=p),

Motives and sample of their perturbed variants:

AAAAAAAAAA AAAAAACCAA AAGGAGACAA AACUUAAAAA AAAAAACAAA AAAAAAAAAA CCAAAAAAUA AAACAACAAA AGUAAACAGU AAGGAAUAAA
CCCCCCCCCC CCGCCUCACC CCCAGGCCCC CCCUCCCCCC CCCACCCCCC CCGGACCCCA CAUCCCCGAC UCCCCCCCAC UCCCCGACCC UCCGCCCCCC
GGGGGGGGGG GAGCAGGUCU GGGGCGGUGG GGGGGGGUUG GGGAGGAGGG GGGGAGGGCG CGGGGGGGGG UGUGGGGUGG CGGGCGGGUG GUGGGGGGGG
UUUUUUUUUU AGUUUAUAUU UUUUUUAUUU UUUUGUUUAG UUUGUUUUUC UUUUUCUACU UUUCUUUUUU CUUUUUUUUU UUAUAUCUUU UUUUUUUUUU


In [6]:
#save to file
fname='artificial_motif_search_dataset.fa'
with open(fname,'w') as f:
    for header,seq in seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

#save explicit negative sequences
from eden.modifier.seq import seq_to_seq, shuffle_modifier
neg_seqs = list(seq_to_seq(seqs, modifier=shuffle_modifier, times=2, order=2))
fname='artificial_motif_search_dataset_negatives.fa'
with open(fname,'w') as f:
    for header,seq in neg_seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

In [1]:
%%time
from sklearn.cluster import MiniBatchKMeans
ca = MiniBatchKMeans(n_clusters=4)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(complexity=2, nbits=14, clustering_algorithm=ca, min_motif_count=1, n_jobs=2)
seqmot.fit(seqs, neg_seqs)
seqmot.save('seqmot')

NameError: name 'seqs' is not defined

In [8]:
for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

0
AAAAAAAAAA 4
AAAAAAAAAG 2
GCCAUAAAA 1
GAAAAGAUAC 1
CUUAAAAAAG 1
AGAAAAAAAG 1
AAUAAAAAA 1
1
GUAGAUGCU 1
GUACGAGCC 1
GGGGGGGGGG 1
CCCCCCCUC 1
CCCCCCCCCC 1
CAUUGCUAAG 1
AAUACCGAGC 1
2
UUUUUUUUUA 1
UAUUUUUUU 1
CUAUUUAUC 1
3
UUAUUAAUA 1
UUAUUAAAUU 1
UAAUAAUUG 1
GGGGAUUAA 1
GAAUUAAAUU 1
CGAUAAUUGC 1
CCGAUUAUAU 1
CAAUUAAUU 1
AUUAAAUAG 1
AAUUACUUAG 1


In [9]:
from eden.motif import SequenceMotif
seqmot2 = SequenceMotif()
seqmot2.load('seqmot')

In [10]:
predictions=seqmot2.predict(seqs, return_list=False)
for p in predictions: print p

1
1
1
0
3
0
0
0
1
0
0
0
1
1
0
1
1
2
1
1
1
1
0
0
0
1
0
0
0
0
0
0
1
1
1
1
1
1
2
1


In [12]:
from eden.motif import SequenceMotif
seqmot2 = SequenceMotif()
seqmot2.load('seqmot')

predictions=seqmot2.predict(seqs, return_list=True)
for p in predictions: print p

[0]
[3]
[3]
[]
[3, 0, 1]
[]
[]
[]
[0]
[]
[]
[]
[0]
[3]
[]
[3]
[0]
[3, 2]
[1]
[2]
[0]
[1]
[]
[]
[]
[1]
[]
[]
[]
[]
[]
[]
[0]
[3]
[1]
[1]
[0]
[0]
[1, 3]
[2]


In [13]:
predictions=seqmot2.transform(seqs, return_match=False)
for p in predictions: print p

[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 1, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 0]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 1, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 1]
[0, 0, 1, 0]


In [38]:
predictions=seqmot2.transform(seqs, return_match=True)
for p in predictions: print p

[[], [(141, 151), (143, 152), (144, 153)], [], [(135, 144)]]
[[], [], [], []]
[[(145, 154), (145, 155), (146, 155), (146, 156), (147, 156)], [], [(195, 204)], []]
[[(188, 197)], [], [], [(0, 9)]]
[[], [], [], []]
[[(72, 81)], [], [], [(145, 155), (258, 268)]]
[[(146, 156)], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[(145, 154), (145, 155), (146, 155), (176, 185)], [], [], [(125, 135)]]
[[], [], [], []]
[[], [(144, 153), (145, 154), (145, 155), (146, 155), (147, 156)], [], []]
[[], [], [], [(146, 155)]]
[[], [], [], []]
[[], [], [], []]
[[], [(145, 154)], [], [(91, 100), (250, 260)]]
[[], [], [], []]
[[], [], [], []]
[[], [], [(147, 157)], [(85, 94)]]
[[], [], [(44, 53)], []]
[[], [], [], [(143, 152), (144, 154), (146, 155)]]
[[], [], [], []]
[[], [], [(146, 156), (147, 156), (147, 157)], []]
[[], [(141, 151), (142, 151)], [], []]
[[], [], [], [(144, 154), (145, 154), (147, 156), (147, 157)]]
[[], [], [], []]
[[], [(53, 63)], [], []]
[[], [], [], []]
[[(7, 17), (29

In [20]:
%%time
from sklearn.cluster import Birch
ca = Birch(threshold=0.1, n_clusters=4, branching_factor=50)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(clustering_algorithm=ca, min_motif_count=2)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 40 ; Features: 1048577 with an avg of 5132 features per instance
Negative data: Instances: 80 ; Features: 1048577 with an avg of 5139 features per instance
Elapsed time: 5.4 secs
model induction: 40 positive instances 5 s
motives extraction: 38 motives 2 s
motives clustering: 4 clusters 2 s
after filtering: 1 motives 1 clusters 0 s
motif model construction: 0 s
updated motif counts: 0 s
0
GGGGGGGGGG 2
CPU times: user 6.18 s, sys: 1.5 s, total: 7.68 s
Wall time: 11.1 s


In [21]:
%%time
from sklearn.cluster import DBSCAN
ca = DBSCAN(eps=0.1, min_samples=3)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(clustering_algorithm=ca, min_motif_count=2)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

Positive data: Instances: 40 ; Features: 1048577 with an avg of 5132 features per instance
Negative data: Instances: 80 ; Features: 1048577 with an avg of 5139 features per instance
Elapsed time: 5.7 secs
model induction: 40 positive instances 6 s
motives extraction: 30 motives 2 s
motives clustering: 1 clusters 0 s
after filtering: 0 motives 0 clusters 0 s
motif model construction: 0 s
updated motif counts: 0 s
CPU times: user 4.11 s, sys: 1.4 s, total: 5.51 s
Wall time: 8.79 s
